In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import pythainlp
from pythainlp.tokenize import sent_tokenize, word_tokenize
import gensim
import string
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import re
pyLDAvis.enable_notebook()
import sefr_cut
sefr_cut.load_model(engine='tl-deepcut-ws1000')

import warnings
warnings.filterwarnings('ignore')

loading model.....
Success


In [2]:
# Add Thai Font
# ที่มา https://www.facebook.com/groups/colab.thailand/permalink/1421960354645985/
import matplotlib as mpl
plt.rcParams['font.family']='K2D'
mpl.font_manager.fontManager.addfont('./fonts/K2D-Regular.ttf')
mpl.rc('font', family='K2D', size=12, weight=200)

# Load and View Dataset

In [5]:
review_df = pd.read_csv('./data/CustomerReviews.csv')
print("Review Dimension (num reviews/num columns) : ",review_df.shape)
review_df.head()

Review Dimension (num reviews/num columns) :  (21, 7)


,Review ID,Restaurant_ID,Restaurant,User,Headline,Review,Rating
0,1,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,7b16469831074f7abc7824745ee75212,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...,5.0
1,2,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,pakkaramonpondej,รสชาติเหมือนทุกสาขา แต่สาขานี้ บริการดี ที่นั่งดี,นึกถึงชาบูญี่ปุ่นยังไงก็ต้อง คิดถึงโมโม่ พาราไ...,5.0
2,3,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,saanowy,ชาบูพรีเมี่ยมสไตล์ญี่ปุ่น เนื้อดีมากกก,มาทานช่วงนี้ สามารถนั่งโต๊ะเดียวกัน หม้อเดียวก...,4.0
3,4,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,ployynp,เนื้อดี ไอติมดี คุ้มค่าดี,ถ้านึกถึงชาบูที่มีเนื้อเน้นๆ ในราคาไม่โหดจนเกิ...,4.0
4,5,352696Px-mo-mo-paradise-เดอะมอลล์-บางกะปิ,Mo-Mo-Paradise (โม โม พาราไดซ์) เดอะมอลล์ บางกะปิ,665a902b335b434ab489f00f2fbb477e,อาหารมีคุณภาพ บริการดีค่ะ,เดินมาหน้าร้านแล้วได้กลิ่นชาบูหอมมาก ๆ ประกอบ...,5.0


# Basic Text Wrangling

In [10]:
# Text Cleansing
# to correct misspelling word and remove the unwanted word or character
correct_word_list = {
    'โมโม่' : 'momo', 'มากก' : 'มาก', 'กกก' : 'ก', 'แวท' : 'vat', 'พิซเซอเรีย' : 'pizzeria', 'พิซซ่า' : 'pizza', 
    'เก้บ' : 'เก็บ', 'ชูชิ' : 'ซูชิ', 'ซึป' : 'ซุป', 'ชาบูชิ' : 'shabushi', 'อารามณ์' : 'อารมณ์', 'ส่งน' : 'ส่วน', 
    'สไลต์' : 'สไลด์', 'ประยุคก์' : 'ประยุกต์', 'ไอติม' : 'icecream', 'พาราไดส์' : 'paradise', 'พาราไดซ์' : 'paradise', 
    'อัพเดท' : 'update', 'นาราย' : 'narai ', 'พรีเมี่ยม' : 'พรีเมียม','บ๋วย' : 'บ๊วย', 'เฟรนไชส์' : 'แฟรนไชส์',
    'บุฟเฟ่ต์่ต์ต์ต์':'บุฟเฟ่ต์','บุฟเฟ่ต์่ต์ต์':'บุฟเฟ่ต์','บุฟเฟ่ต์ต์':'บุฟเฟ่ต์',
    'บุพเฟ่' : 'บุฟเฟ่ต์','บุฟเฟ่' : 'บุฟเฟ่ต์','บุฟเฟ' : 'บุฟเฟ่ต์', 'บุฟเฟต' : 'บุฟเฟ่ต์',
    'รสชาต' : 'รสชาติ'
}

def do_text_preprocessing(text):
    # 1. correct some misspelling in text
    for old,new in correct_word_list.items():
        text = text.replace(old, new)
        
    # 2. remove punctuations character in text
    text = re.sub(r'[ๆฯ!#$&%\"\'()*+,-./:;<=>?@\[\]\\^_`{}|~]',' ', text)

    # 3. remove digit character in text
    text = re.sub(r'\d',' ', text)
    
    return text.strip()

# the review text is the review's headline and contents
review_df['review_text'] = review_df.apply(lambda x: do_text_preprocessing(x['Headline'] + ' ' + x['Review']), axis=1)
print(review_df['review_text'])

0     ที่สำคัญของร้านนี้คือบริการดีมากพนักงานน่ารักส...
1     รสชาติิเหมือนทุกสาขา แต่สาขานี้ บริการดี ที่นั...
2     ชาบูพรีเมียมสไตล์ญี่ปุ่น เนื้อดีมากก มาทานช่วง...
3     เนื้อดี icecreamดี คุ้มค่าดี ถ้านึกถึงชาบูที่ม...
4     อาหารมีคุณภาพ บริการดีค่ะ เดินมาหน้าร้านแล้วได...
5     กินให้อิ่ม ที่ momo ร้านบุฟเฟ่ต์่ต์ ชาบูแนวญี่...
6     ชาบู   สุกกี้สไลด์ญี่ปุ่นที่แท้ทรู  Number    ...
7     ร้านชาบูแฟรนไชส์ ที่ราคาเหมาะสมกับคุณภาพ ถือว่...
8     อร่อยคุ้ม ของทานเล่นเยอะมาก มา านที่ขาบูชิต้อง...
9     ทานอีก ชอบกุ้งทอดเทมปุระ ใครชอบกุ้งทอดเทมปุระ ...
10    บุฟเฟ่ต์่ต์ต์คุ้ม ในห้าง กลับมาupdateราคาshabu...
11    เปลี่ยนจากกุ้งขาดสารอาหารเป็นกุ้งปอกเปลือกแล้ว...
12    อิ่ม อร่อย ฝุดฝุด เมื่อหลายวันก่อนนัดหาข้าวทาน...
13    จัดให้เต็มคราบกับชั่วโมงครึ่ง Shabushi บอกตรง ...
14    ชาบู shabushi สวัสดีครับวันนี้จะขอมารีวิวร้านs...
15    บุฟเฟ่ต์ต์น้ำเดือด สายพานร่อน shabushi สาขาเดอ...
16    บุฟเฟ่ต์่ต์ต์ชาบูและpizzaไม่อั้นในราคา     บาท...
17    ร้านบุฟเฟ่ต์่ต์ต์ ราคามิตรภาพ อยู่ชั้น  ติ

In [12]:
# word segmentation and remove the common words(stop words)
thai_stopwords = list(pythainlp.corpus.thai_stopwords())
unwanted_word = ['(', ')' ,'😆','🤣','"','','%','\u200b','::',
                 'ร้าน','บาท','สำหรับ','ชื่อ','ทาน','ดี','กิน','อาหาร',
                 'ดู','คน','ตัว','ลอง','ตอน','เลือก','ใจ','ที่']
remove_word_list = thai_stopwords + unwanted_word

def do_word_tokenization(text):
    word_list = []
    for sentence in sent_tokenize(text, engine='whitespace+newline'):
        for words in sefr_cut.tokenize(sentence,k=100):
            for word in words:
                if len(word) >1 and word not in remove_word_list:
                    word_list.append(word)
    return word_list

%time review_df['review_token'] = review_df['review_text'].apply(do_word_tokenization)
print(review_df['review_token'])

CPU times: user 32.5 s, sys: 2.76 s, total: 35.2 s
Wall time: 28 s
0     [บริการ, พนักงาน, น่ารัก, สะอาด, สะอ้าน, ใส่, ...
1     [รสชาติิ, เหมือน, สาขา, สาขา, บริการ, นั่ง, นึ...
2     [ชาบูพรีเมียม, สไตล์, ญี่ปุ่น, เนื้อ, มากก, นั...
3     [เนื้อ, icecream, คุ้ม, ค่า, นึก, ชาบู, เนื้อ,...
4     [ภาพ, บริการ, เดิน, หน้า, กลิ่น, ชาบูหอม, โปรบ...
5     [อิ่ม, momo, บุฟเฟ่ต์่ต์, ชา, บู, แนว, ญี่ปุ่น...
6     [ชาบู, สุก, กี้, สไลด์, ญี่ปุ่น, ทรู, Number, ...
7     [ชาบูแฟรนไชส์, ราคา, เหมาะสม, คุณภาพ, คุ้ม, น้...
8     [อร่อย, คุ้ม, เล่น, าน, ขา, บูชิ, หิว, งั้น, ค...
9     [ชอบ, กุ้ง, ทอด, เทมปุระ, ชอบ, กุ้ง, ทอด, เทมป...
10    [บุฟเฟ่ต์่ต์ต์, คุ้ม, ห้าง, update, ราคา, shab...
11    [กุ้ง, สาร, กุ้ง, ปอก, เปลือก, รู้, ห่างหาย, s...
12    [อิ่ม, อร่อย, ฝุดฝุด, นัด, หา, ข้าว, เดอะแกงค์...
13    [เต็ม, คราบ, ชั่วโมง, ครึ่ง, Shabushi, บุฟเฟ่ต...
14    [ชาบู, shabushi, สวัสดี, รี, วิว, shabushi, บุ...
15    [บุฟเฟ่ต์ต์, น้ำ, เดือด, สายพานร่อน, shabushi,...
16    [บุฟเฟ่ต์่ต์ต์, ชาบู, pizza, อั

In [15]:
# word segmentation and remove the common words(stop words)
thai_stopwords = list(pythainlp.corpus.thai_stopwords())
unwanted_word = ['(', ')' ,'😆','🤣','"','','%','\u200b','::',
                 'ร้าน','บาท','สำหรับ','ชื่อ','ทาน','ดี','กิน','อาหาร',
                 'ดู','คน','ตัว','ลอง','ตอน','เลือก','ใจ','ที่']
remove_word_list = thai_stopwords + unwanted_word

def do_word_tokenization(text):
    word_list = []
    for sentence in sent_tokenize(text, engine='whitespace+newline'):
        for words in sefr_cut.tokenize(sentence,k=100):
            for word in words:
                if len(word) >1 and word not in remove_word_list:
                    word_list.append(word)
    return word_list

%time review_df['review_token'] = review_df['review_text'].apply(do_word_tokenization)
print(review_df['review_token'])

CPU times: user 33.7 s, sys: 2.91 s, total: 36.7 s
Wall time: 29.3 s
0     [บริการ, พนักงาน, น่ารัก, สะอาด, สะอ้าน, ใส่, ...
1     [รสชาติิ, เหมือน, สาขา, สาขา, บริการ, นั่ง, นึ...
2     [ชาบูพรีเมียม, สไตล์, ญี่ปุ่น, เนื้อ, มากก, นั...
3     [เนื้อ, icecream, คุ้ม, ค่า, นึก, ชาบู, เนื้อ,...
4     [ภาพ, บริการ, เดิน, หน้า, กลิ่น, ชาบูหอม, โปรบ...
5     [อิ่ม, momo, บุฟเฟ่ต์่ต์, ชา, บู, แนว, ญี่ปุ่น...
6     [ชาบู, สุก, กี้, สไลด์, ญี่ปุ่น, ทรู, Number, ...
7     [ชาบูแฟรนไชส์, ราคา, เหมาะสม, คุณภาพ, คุ้ม, น้...
8     [อร่อย, คุ้ม, เล่น, าน, ขา, บูชิ, หิว, งั้น, ค...
9     [ชอบ, กุ้ง, ทอด, เทมปุระ, ชอบ, กุ้ง, ทอด, เทมป...
10    [บุฟเฟ่ต์่ต์ต์, คุ้ม, ห้าง, update, ราคา, shab...
11    [กุ้ง, สาร, กุ้ง, ปอก, เปลือก, รู้, ห่างหาย, s...
12    [อิ่ม, อร่อย, ฝุดฝุด, นัด, หา, ข้าว, เดอะแกงค์...
13    [เต็ม, คราบ, ชั่วโมง, ครึ่ง, Shabushi, บุฟเฟ่ต...
14    [ชาบู, shabushi, สวัสดี, รี, วิว, shabushi, บุ...
15    [บุฟเฟ่ต์ต์, น้ำ, เดือด, สายพานร่อน, shabushi,...
16    [บุฟเฟ่ต์่ต์ต์, ชาบู, pizza, 

# Topic Models with Gensim
Gensim เป็น framework สำหรับทำ Topic Model, Text Similarity, Sematic Analytics และ Text Summarization โดย Gensim มีความยืดหยุ่นกว่า scikit-learn

In [ ]:
review_token_list = review_df['review_token']
dictionary = gensim.corpora.Dictionary(review_token_list)

In [ ]:
print(str(dictionary.token2id.keys()))

In [ ]:
num_topics = 5
chunksize = 4000 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 100
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
# temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in review_token_list]
word_frequencies = [[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]

%time model = gensim.models.LdaModel(\
    corpus=gensim_corpus, \
    id2word=id2word, \
    chunksize=chunksize,\
    alpha='auto',\
    eta='auto',\
    iterations=iterations,\
    num_topics=num_topics,\
    passes=passes,\
    eval_every=eval_every)

In [ ]:
pyLDAvis.gensim_models.prepare(model, gensim_corpus, dictionary)

In [ ]:
def get_keyword(x):
  new_x = str(c[x])
  return new_x

def get_topic(x):
  new_x = [i[0] for i in x]
  return new_x

list_topic = list()
for i in range(0,num_topics):
  print('topic:',i)
  list_topic.append(set(get_topic(model.show_topic(i,20))))
  print(list_topic[i])

In [ ]:
result = []
topn = 15

for n in range(num_topics):
    temp_df = pd.DataFrame(model.get_topic_terms(n, topn=topn), columns=['word_id','prob'])
    temp_df['topic'] = n
    result.append(temp_df)

topic_terms_df = pd.concat(result)
topic_terms_df['word'] = topic_terms_df['word_id'].apply(lambda x: dictionary.get(x))
topic_terms_df.head()

In [ ]:
c = list()
for i in range(0,num_topics):
  a = list_topic[i]
  b = set()
  for j in range(0,num_topics):
    if i != j:
      b = b.union(list_topic[j])
  a = a-b
  c.append(list(a))
  print(i,a)

In [ ]:
review_df['topic'] = review_df['review_token'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x))[0][0])
review_df['score'] = review_df['review_token'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x))[0][1])
review_df['topic_name'] = review_df['review_token'].apply(lambda x: model.show_topic(model.get_document_topics(dictionary.doc2bow(x))[0][0]))
review_df['topic_name2'] = review_df['topic_name'].apply(lambda x:get_topic(x))
review_df['keyword'] = review_df['topic'].apply(lambda x:get_keyword(x))

In [ ]:
print(review_df['topic_name2'])

In [ ]:
review_df[['Restaurant','review_text','topic','score','topic_name2','keyword']]